# Point Cloud Based 3D Object Detection Inference using ONNX Runtime


## Inference using Pre-Compiled Models
In this example notebook, we describe how to use a pre-trained Point Cloud Based 3D Object Detection model for inference using the ***ONNX Runtime interface***.
   - Model is based on point pillars [Link](https://arxiv.org/abs/1812.05784) method.
   - Model is trained for only one class, which is 'car'
   - We perform inference on a few sample point clouds
   - We also describe the input preprocessing and output postprocessing steps, demonstrate how to collect various benchmarking statistics and how to visualize the data.


## Point Cloud Based 3D Object Detection  

3D Object Detection is a popular computer vision algorithm used in many applications such as Person Detection and Vehicle detection. 3D object information provides better understanding surrounding and hence helps in precise path planning. 3D object detection on lidar data outperforms than image data. 

## ONNX Runtime based Work flow

The diagram below describes the steps for ONNX Runtime based workflow. 

Note:
- The user needs to compile models(sub-graph creation and quantization) on a PC to generate model artifacts.
    - For this notebook we use pre-compiled models artifacts
- The generated artifacts can then be used to run inference on the target.
- Users can run this notebook as-is, only action required is to select a model. 

<img src=docs/images/onnx_work_flow_2.png width="400">

In [ ]:
import os
import cv2
import numpy as np
import ipywidgets as widgets
from scripts.utils import get_eval_configs

prebuilt_configs, selected_model_id = get_eval_configs('detection_3d','onnxrt', 
                                                       num_quant_bits = 8, 
                                                       last_artifacts_id = None,
                                                       experimental_models=True)
display(selected_model_id)

In [ ]:
print(f'Selected Model: {selected_model_id.label}')
config = prebuilt_configs[selected_model_id.value]
config['session'].set_param('model_id', selected_model_id.value)
config['session'].start()

## Create the model using the stored artifacts
<div class="alert alert-block alert-warning">
<b>Warning:</b> It is recommended to use the ONNX Runtime APIs in the cells below without any modifications.
</div>

In [ ]:
import onnxruntime as rt

onnx_model_path = config['session'].get_param('model_file')
delegate_options = {}
so = rt.SessionOptions()
delegate_options['artifacts_folder'] = config['session'].get_param('artifacts_folder')

EP_list = ['TIDLExecutionProvider','CPUExecutionProvider']
sess = rt.InferenceSession(onnx_model_path ,providers=EP_list, provider_options=[delegate_options, {}], sess_options=so)

input_details = sess.get_inputs()
output_details = sess.get_outputs()

## Sample Point Cloud data
Prepare the list of point cloud data to be processed. Similar list has to prepared for image data and clibration data. 
    

### Disclaimer ::
  - We use one sample point cloud data, corresponding image data and calibration information from https://github.com/azureology/kitti-velo2cam/blob/master/readme.md. Currently only one lidar frame is hosted there. This data is used in main processing loop.
  - This point cloud is from Kitti data set, and user of this jupyter notebook is assumed to have agreed to all the terms and conditions for usages of this dataset content. 
  - Refer "http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=3d" for license of usages for Kitti 3d-od dataset.
  


In [ ]:
    
image_files = [
    ('000007.png')
]

point_cloud_files =[
    ('000007.bin'),
]

calib_files =[
    ('000007.txt'),
]


## Initialize configuration parameters required for point cloud pre-processing

Input point cloud is segregated in 3D bins called as voxels.Features are computed for each voxels. pre-procssing step (voxelization) needs these configuration parameters



In [ ]:
voxelization_config_params={}

# voxel property 
voxelization_config_params['min_x'] = 0
voxelization_config_params['max_x'] = 69.120
voxelization_config_params['min_y'] = -39.680
voxelization_config_params['max_y'] = 39.680
voxelization_config_params['min_z'] = -3.0
voxelization_config_params['max_z'] = 1.0
voxelization_config_params['voxel_size_x']= 0.16
voxelization_config_params['voxel_size_y']= 0.16
voxelization_config_params['voxel_size_z']= 4.0
voxelization_config_params['num_voxel_x'] = (voxelization_config_params['max_x'] - 
                                             voxelization_config_params['min_x'])/voxelization_config_params['voxel_size_x']

voxelization_config_params['num_voxel_y'] = (voxelization_config_params['max_y'] - 
                                             voxelization_config_params['min_y'])/voxelization_config_params['voxel_size_y']

# network property has to align with below parameters
voxelization_config_params['max_points_per_voxel'] = 32
voxelization_config_params['nw_max_num_voxels']    = 10000
voxelization_config_params['num_feat_per_voxel']   = 10
voxelization_config_params['num_channels']         = 64


### Pre-processing 
  - Point cloud data will have data from all around 360 degree, hence consider the point cloud 
    which are in front of camera view (`align_img_and_pc` API)
  - Perform voxelization on a set of lidar data mentioned in the list 'point_cloud_files'. (`voxelization` API)
  - Feature computation for each voxel. (`voxelization` API)

### Inference
  - Real TIDL inference 
  
### Post-processing and Visualization
 - Object Detection models return results as a list (i.e. `numpy.ndarray`) with length of 9. Each element in this list contains, the detected object class ID, the probability of the detection and the 3d bounding box co-ordinates.
 - We use the `boxes3d_to_corners3d_lidar()` API to identify 8 corners of 3D box in 3 dimensional space.
 - `draw_lidar_bbox3d_on_img()` API is used for drawing lines between two corner
 - Then, in this notebook, we use *matplotlib* to plot the original images and the corresponding results.


In [ ]:
import tqdm
import matplotlib.pyplot as plt
from PIL import Image
import urllib

from scripts.utils_lidar import boxes3d_to_corners3d_lidar
from scripts.utils_lidar import draw_lidar_bbox3d_on_img
from scripts.utils_lidar import voxelization
from scripts.utils_lidar import align_img_and_pc

fig = plt.figure(figsize=(10,10))

#Non Empty Voxel features. Populated by the `voxelization` API
input0 = np.zeros((1, 
                   voxelization_config_params['num_feat_per_voxel'], 
                   voxelization_config_params['max_points_per_voxel'], 
                   voxelization_config_params['nw_max_num_voxels']),
                  dtype='float32')

#Voxel index in 2D convas. 2D convas where voxel features gets scattered. Populated by the `voxelization` API
input1 = np.zeros((1, voxelization_config_params['num_channels'], 
                   voxelization_config_params['nw_max_num_voxels']),
                  dtype='int32')

#2D canvas initialized with zero
input2 = np.zeros((1, voxelization_config_params['num_channels'], 
                   (int)(voxelization_config_params['num_voxel_x']*voxelization_config_params['num_voxel_y'])),
                  dtype='float32')

# In onnx rt flow double data read happens hence that can be discounted in DDR read measurement
input_size_float = input0.size * input0.itemsize + input1.size * input1.itemsize + input2.size * input2.itemsize

ax = []

for num in tqdm.trange(len(image_files)):
    image_file = image_files[num]
    point_cloud_file = point_cloud_files[num]
    calib_file       = calib_files[num]

    # read int8 image data
    req = urllib.request.urlopen('https://raw.githubusercontent.com/azureology/kitti-velo2cam/master/data_object_image_2/testing/image_2/' + image_file)
    arr = np.asarray(bytearray(req.read()), dtype=np.int8)
    img = cv2.imdecode(arr, -1) # 'Load it as it is'
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # read point float32 cloud data
    req = urllib.request.urlopen('https://raw.githubusercontent.com/azureology/kitti-velo2cam/master/data_object_velodyne/testing/velodyne/' + point_cloud_file)
    point_cloud_raw = np.fromstring(req.read(), dtype=np.float32).reshape(-1, 4)

    # read calib text data
    req = urllib.request.urlopen('https://raw.githubusercontent.com/azureology/kitti-velo2cam/master/testing/calib/' + calib_file)
    calib_line = []
    for line in req:
        calib_line.append(line.decode("utf-8"))

    ### Pre-processing    
    # point cloud data will have data from all around 360 degree, hence consider the point cloud 
    #which are in front of camera view
    lidar_data, lidar2img_rt = align_img_and_pc(img, point_cloud_raw, calib_line)

    voxelization(lidar_data,voxelization_config_params,input0[0],input1[0])
    
    ### Inference
    output = sess.run(None, {input_details[0].name: input0, input_details[1].name: input1, input_details[2].name: input2})
    selected_objs = output[0][0][0][(output[0][0][0][:,1]>0.3)]
    ### Post-processing
    corners = boxes3d_to_corners3d_lidar(selected_objs[:,2:])
    img = draw_lidar_bbox3d_on_img(corners,img,lidar2img_rt)
    
    ax.append(fig.add_subplot(len(image_files), 1, num+1) )
    plt.imshow(img)

plt.show()


## Plot Inference benchmarking statistics
 - During model execution several benchmarking statistics such as timestamps at different checkpoints, DDR bandwidth are collected and stored. 
 - The `get_TI_benchmark_data()` function can be used to collect these statistics. The statistics are collected as a dictionary of `annotations` and corresponding markers.
 - We provide the utility function plot_TI_benchmark_data to visualize these benchmark KPIs.

<div class="alert alert-block alert-info">
<b>Note:</b> The values represented by <i>Inferences Per Second</i> and <i>Inference Time Per Image</i> uses the total time taken by the inference except the time taken for copying inputs and outputs. In a performance oriented system, these operations can be bypassed by writing the data directly into shared memory and performing on-the-fly input / output normalization.
</div>


In [ ]:
from scripts.utils import plot_TI_performance_data, plot_TI_DDRBW_data, get_benchmark_output, print_soc_info
stats = sess.get_TI_benchmark_data()
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10,5))
plot_TI_performance_data(stats, axis=ax)
plt.show()

tt, st, rb, wb = get_benchmark_output(stats)

rb_exclude_float_ip = rb - input_size_float # saving because of onnxrt double input accounting

print_soc_info()
print(f'{selected_model_id.label} :')
print(f' Inferences Per Second    : {1000.0/tt :7.2f} fps')
print(f' Inference Time Per Image : {tt :7.2f} ms')
print(f' DDR usage Per Lidar Frame: {rb+ wb : 7.2f} MB')
print(f' DDR usage Per Lidar Frame excluding float input      : {rb_exclude_float_ip+ wb : 7.2f} MB')